# Create a mural from sample .json

This notebook demonstrates how to create a new mural containing widgets defined in a .json file.

A sample .json file containing the contents of a sample mural is available ere: [Sample .json file](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-02_overlap-inside-rectangle.json)

The following image shows what the sample mural looks like:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-02_overlap-inside-rectangle.png" width="50%" title="Sample mural" />

## Step 1: Read sample .json file

In [143]:
import urllib
import json

url = "https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/murals/sample-02_overlap-inside-rectangle.json"
response = urllib.request.urlopen( url )
encoding = response.info().get_content_charset( "utf8" )
widgets_arr = json.loads( response.read().decode( encoding ) )

In [144]:
print( json.dumps( widgets_arr, indent = 3 ) )

[
   {
      "height": 382.750001220703,
      "id": "0-1650991681040",
      "width": 608.2500024414062,
      "x": 302.94,
      "y": 314.78,
      "shape": "square",
      "style": {
         "backgroundColor": "#FFFFFF00",
         "strokeColor": "#000000FF",
         "strokeSize": 3,
         "strokeStyle": "solid"
      },
      "type": "shape"
   },
   {
      "height": 138,
      "id": "0-1650991688310",
      "title": "",
      "width": 138,
      "x": 233.94,
      "y": 156.77999999999997,
      "shape": "rectangle",
      "style": {
         "backgroundColor": "#CB8CF9FF",
         "bold": false,
         "font": "proxima-nova",
         "fontSize": 23,
         "italic": false,
         "strike": false,
         "textAlign": "center",
         "underline": false,
         "border": false
      },
      "text": "This is outside",
      "type": "sticky note"
   },
   {
      "height": 138,
      "id": "0-1650991701198",
      "width": 138,
      "x": 564.07,
      "y": 239.28

## Step 2: Collect the workspace ID and room ID where the mural will go

You can find the workspace ID and the room ID in the url of a room.

Room urls look something like this:

```
https://app.mural.co/t/<workspace>/r/<id>
```

What you need to pass to the MURAL API is before and after the `/r/`: the \<workspace> and the \<id>.

For example, if you have a room with this url:

```
https://app.mural.co/t/teamideas1234/r/1234567890123
```

Then, the workspace ID is "teamideas1234" and the room ID is "1234567890123".

In [101]:
g_workspace_id = ""

In [102]:
g_room_id = ""

## Step 3: Collect your OAuth token

In [131]:
g_auth_token = ""

## Step 4: Create a blank mural

In [20]:
import requests

def createMural( auth_token, workspace_id, room_id, title ):
    # https://developers.mural.co/public/reference/createmural
    url = "https://app.mural.co/api/public/v1/murals"
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = { "workspaceId" : workspace_id, 
              "roomId"      : int( room_id ),  # If this is not a number, call fails
              "title"       : title }
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    if "id" not in response_json["value"]:
        print( "No id returned" )
        return None
    return response_json["value"]["id"]

In [22]:
title = "Created using the API!"
mural_id = createMural( "dummy", g_workspace_id, g_room_id, title )
print( mural_id )

UNAUTHORIZED You don't have the required permissions to access this endpoint.
None


In [23]:
title = "Created using the API!"
mural_id = createMural( g_auth_token, "dummy", g_room_id, title )
print( mural_id )

WORKSPACE_NOT_FOUND The workspace was not found or does not exist.
None


In [24]:
title = "Created using the API!"
mural_id = createMural( g_auth_token, g_workspace_id, g_room_id, title )
print( mural_id )

teamideas1234.1651006118176


## Step 5: Put all the widgets into the mural

In [141]:
import re
import copy

def putWidget( auth_token, mural_id, widget ):
    # https://developers.mural.co/public/reference/createstickynote
    # https://developers.mural.co/public/reference/createshapewidget
    widget_type = widget["type"]
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/" + re.sub( "\s+", "-", widget_type )
    headers = { #"Accept"        : "application/json", 
                "Accept"        : "vnd.mural.preview",
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = copy.deepcopy( widget )
    if "id" in parms:
        del parms["id"]
    if "type" in parms:
        del parms["type"]
    #if "style" in parms:
    #    del parms["style"]
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + widget["id"] + " ] " + msg )
    else:
        print( "[ " + widget["id"] + " ] Success" )

In [142]:
for widget in widgets_arr:
    putWidget( g_auth_token, mural_id, widget )

[ 0-1650991681040 ] Success
[ 0-1650991688310 ] Success
[ 0-1650991701198 ] Success
[ 0-1650991717299 ] Success
